In [106]:
import pandas as pd
import numpy as np
from datetime import datetime

In [107]:
data_path='D:\PythonDev\MyQuantFinProject\Data\AllThai-Oct22-Jan23.csv'


trade_plan_path='MasterConfigData\ListTradePlan.xlsx'
trade_sheet='July22-Now'

asset_comm_path='MasterConfigData\AssetInfo.xlsx'

result_path=f'ReportData\ReportTestPort_{trade_sheet}-SMALL.xlsx'


In [108]:
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()

print(list_symbols)
df.info()
print(df.tail())


['ASP-T12' 'ASP-SME' 'ASP-THEQ' 'K-MIDSMALL' 'KFTHAISM' 'KKP-ACT-EQ-A'
 'KFDYNAMIC' 'K-SET50' 'KKP-SM-CAP' 'KKP-TQG' 'TISCOFLEXP' 'TLMSEQ-A'
 'TLEQ' 'TISCOMS-A' 'TSF-A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6757 entries, 0 to 6756
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  6757 non-null   object        
 1   date    6757 non-null   datetime64[ns]
 2   price   6757 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 158.5+ KB
     symbol       date  price
6752  TSF-A 2023-01-23  85.10
6753  TSF-A 2023-01-24  85.58
6754  TSF-A 2023-01-25  85.08
6755  TSF-A 2023-01-26  84.25
6756  TSF-A 2023-01-27  85.30


In [112]:
# Fix Symbol as you want if not compare all in excel
list_symbols=['KKP-SM-CAP','KKP-ACT-EQ-A','KKP-TQG','TISCOFLEXP','TISCOMS-A','TSF-A','ASP-T12','ASP-SME','ASP-THEQ']
# list_symbols=['KKP-SM-CAP']

dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

dfNoComissionInfo= [ item for item in list_symbols if item not in list(dfAssetInfo.index)]
if  len(dfNoComissionInfo)>0:
  raise Exception(f"{dfNoComissionInfo} didn't found commision info in {asset_comm_path}")
else:
  dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]
  print("Commmision info")
  print(dfAssetInfo)  

Commmision info
              buy_comm_pct  sell_comm_pct
symbol                                   
KKP-SM-CAP            1.00            0.0
KKP-ACT-EQ-A          0.50            0.0
KKP-TQG               0.50            0.0
TISCOFLEXP            1.00            0.0
TISCOMS-A             1.00            0.0
TSF-A                 1.05            0.0
ASP-T12               1.25            0.0
ASP-SME               1.25            0.0


In [113]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})



#we have to add last reord 100 % as of last dat in Amibroker database
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()
all_portion=0
if buy_sum_act<sell_sum_act:
  raise Exception("Not allow Buy <= Sell")   
else:
    if buy_sum_act>sell_sum_act:
       print("=============No explicite lasst sell , we will append SellAll-At-LastPrice=============")
       sellAllAtNow=buy_sum_act-sell_sum_act

       dfPlan=pd.concat([dfPlan,pd.DataFrame(columns=list(dfPlan.columns), data=[ [df['date'].max(),sellAllAtNow,'s']])]) 
       dfPlan=dfPlan.reset_index(drop=True)

    tradeDateList= [ d for d in list(dfPlan['trade_date']) ]    
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    
print("================================================================================================")
print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")
print(dfPlan)


=============No explicite lasst sell , we will append SellAll-At-LastPrice=============
portion = 100.0 on
No Buy/Sell action
b    5
s    1
dtype: int64
  trade_date  portion_pct action
0 2022-10-20         20.0      b
1 2022-11-01         20.0      b
2 2022-11-25         20.0      b
3 2022-12-30         20.0      b
4 2023-01-06         20.0      b
5 2023-01-27        100.0      s


In [114]:
TotalBudget=100000
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception(f"Your Budget {TotalBudget} is  not proportiona;  Portion with {all_portion}")
else:
  print(f"Your Budget {TotalBudget} is  proportional  Portion with {all_portion}")

Your Budget 100000 is  proportional  Portion with 100.0


In [115]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty


In [116]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    
    
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    noPlanedTrade= len(tradeDateList)
    noActualTrade=len(dfBS)
    if noPlanedTrade==len(dfBS):
    
        print(f"{symbol} Buy/Sell ={noPlanedTrade}")
        dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')

        dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
        dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
        dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)

        dfBS=dfBS.drop(columns=['trade_date'])
        #print(dfBS)
    
        print("================Buy=================")
        buyDF=dfBS.query("action=='b'")    
        # cal qury and actual value and  total_val and totl qury
        buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
        buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
        buyDF['action_value']=buyDF['value']
        buyDF['action_qty']=buyDF['qty']
        print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        TotalQty=buyDF['qty'].sum()
        TotalValue=round(buyDF['value'].sum(),2)
        AvgPrice=round(TotalValue/TotalQty,4)
        # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")

        print("=================Sell=================")
        sellDF=dfBS.query("action=='s'")  
        sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
        sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
        sellDF['action_value']=sellDF['value']*-1
        sellDF['action_qty']=sellDF['qty']*-1
        print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])

        SellTotalQty=sellDF['qty'].sum()
        SellTotalValue=round(sellDF['value'].sum(),2)
        SellAvgPrice=round(SellTotalValue/SellTotalQty,4)

        # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")

        PL=round(SellTotalValue-TotalValue,0)
        PCT_PL=round(PL/TotalValue*100,2)

        summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                      'PL':PL,'Percent_PL':PCT_PL
                     }
        dfSummary=dfSummary.append(summary_data,ignore_index=True)

        dfTran=pd.concat([buyDF,sellDF],axis=0)
        dfTran=dfTran.sort_values(by=['date'])
        dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

        dict_dfTran[symbol]=dfTran
    else:
        print(f"{symbol} has been exclued from trading process due to{noActualTrade}<> {noPlanedTrade}")


KKP-SM-CAP Buy/Sell =6
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-10-20          17.17  1164.822365  20000.0         20.0
1 2022-11-01          17.35  1152.737752  20000.0         20.0
2 2022-11-25          17.58  1137.656428  20000.0         20.0
3 2022-12-30          17.23  1160.766106  20000.0         20.0
4 2023-01-06          17.25  1159.420290  20000.0         20.0
=================Sell=================
        date  price_on_comm         qty         value  portion_pct
5 2023-01-27          17.21  5775.40294  99394.684597        100.0
KKP-ACT-EQ-A Buy/Sell =6
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2022-10-20          18.55  1078.167116  20000.0         20.0
1 2022-11-01          18.88  1059.322034  20000.0         20.0
2 2022-11-25          18.82  1062.699256  20000.0         20.0
3 2022-12-30          19.13  1045.478306  20000.0         20.0
4 2023-01-06

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\526447759.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [117]:
for symbol,tranDF in dict_dfTran.items():
  tranDF['net_value']=tranDF['action_value'].cumsum(axis = 0) 
  tranDF['net_qty']=tranDF['action_qty'].cumsum(axis = 0)  
  tranDF['avg_price']=round( tranDF['net_value']/tranDF['net_qty'],4)
  tranDF=tranDF.sort_values(by=['date'])
  print(tranDF[['date' ,'action','price_on_comm','qty','net_qty','value','net_value','avg_price']])

        date action  price_on_comm          qty      net_qty    value  \
0 2022-10-20      b          17.17  1164.822365  1164.822365  20000.0   
1 2022-11-01      b          17.35  1152.737752  2317.560117  20000.0   
2 2022-11-25      b          17.58  1137.656428  3455.216545  20000.0   
3 2022-12-30      b          17.23  1160.766106  4615.982650  20000.0   
4 2023-01-06      b          17.25  1159.420290  5775.402940  20000.0   
5 2023-01-27      s          17.21  5775.402940     0.000000  99395.0   

   net_value  avg_price  
0    20000.0    17.1700  
1    40000.0    17.2595  
2    60000.0    17.3650  
3    80000.0    17.3311  
4   100000.0    17.3148  
5      605.0        inf  
        date action  price_on_comm          qty      net_qty     value  \
0 2022-10-20      b          18.55  1078.167116  1078.167116   20000.0   
1 2022-11-01      b          18.88  1059.322034  2137.489150   20000.0   
2 2022-11-25      b          18.82  1062.699256  3200.188406   20000.0   
3 2022-12-

In [118]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)

for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_27252\1780546000.py:8: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Successfully


D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [119]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
1,KKP-ACT-EQ-A,100000.0,18.8936,102256.65,19.32,2257.0,2.26
4,TISCOMS-A,100000.0,20.6677,102188.63,21.12,2189.0,2.19
5,TSF-A,100000.0,83.5954,102039.09,85.30,2039.0,2.04
7,ASP-SME,100000.0,17.7959,101540.48,18.07,1540.0,1.54
2,KKP-TQG,100000.0,8.8031,101214.49,8.91,1214.0,1.21
3,TISCOFLEXP,100000.0,60.6773,101125.08,61.36,1125.0,1.12
6,ASP-T12,100000.0,10.9365,100398.15,10.98,398.0,0.40
0,KKP-SM-CAP,100000.0,17.3148,99394.68,17.21,-605.0,-0.60
